In [1]:
import pandas as pd
import os
import pyarrow
import openpyxl

import os
import pyarrow
import openpyxl
import pickle
from typing import Dict

from exgauster.utils import DataOfExgauster
from dataclasses import dataclass
from typing import Optional
import exgauster.trained_booster



C:\tools\miniconda3\envs\stal\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
joined_m1 = pd.read_parquet('m1_answers_noresampled.parquet')
joined_m1_resampled = pd.read_parquet('submission_2_only_M1==1.parquet')
joined_m3 = pd.read_parquet('m3_answers_noresampled.parquet')
intervals = pd.read_excel(os.path.join('..', 'Датасеты', 'test_intervals.xlsx'))

In [14]:
intervals

,Unnamed: 0,start,finish,machine,tm
0,97,2022-01-01 10:54:53,2022-01-01 16:43:39,NaN,NaN
1,118,2022-01-02 12:26:42,2022-01-02 15:16:06,NaN,NaN
2,126,2022-01-04 12:06:07,2022-01-04 15:04:34,NaN,NaN
3,163,2022-01-04 20:13:57,2022-01-05 00:19:40,NaN,NaN
4,0,2022-01-07 09:05:16,2022-01-07 14:05:15,NaN,NaN
...,...,...,...,...,...
184,66,2023-04-07 15:36:42,2023-04-07 19:27:38,NaN,NaN
185,138,2023-04-09 19:55:43,2023-04-09 23:12:53,NaN,NaN
186,67,2023-04-10 18:08:25,2023-04-10 21:47:02,NaN,NaN
187,68,2023-04-18 02:28:43,2023-04-18 06:38:47,NaN,NaN


In [3]:
for idx, row in intervals.iterrows():
    start_date = row['start']
    end_date = row['finish']
    print(idx, start_date, end_date)


0 2022-01-01 10:54:53 2022-01-01 16:43:39
1 2022-01-02 12:26:42 2022-01-02 15:16:06
2 2022-01-04 12:06:07 2022-01-04 15:04:34
3 2022-01-04 20:13:57 2022-01-05 00:19:40
4 2022-01-07 09:05:16 2022-01-07 14:05:15
5 2022-01-08 08:06:21 2022-01-08 11:06:14
6 2022-01-09 03:32:20 2022-01-09 07:06:45
7 2022-01-10 23:27:44 2022-01-11 03:41:15
8 2022-01-16 20:01:07 2022-01-16 22:50:17
9 2022-01-19 02:24:42 2022-01-19 07:29:32
10 2022-01-19 15:03:18 2022-01-19 19:30:07
11 2022-01-24 05:57:27 2022-01-24 09:22:48
12 2022-01-24 23:56:59 2022-01-25 02:51:29
13 2022-01-29 23:33:45 2022-01-30 04:06:42
14 2022-02-03 23:51:16 2022-02-04 02:47:56
15 2022-02-06 18:05:24 2022-02-06 23:44:27
16 2022-02-12 16:37:03 2022-02-13 09:42:17
17 2022-02-24 14:52:35 2022-02-24 18:58:49
18 2022-02-25 03:44:52 2022-02-25 08:15:03
19 2022-03-03 06:01:43 2022-03-03 12:01:24
20 2022-03-06 22:18:14 2022-03-07 03:37:22
21 2022-03-11 06:06:36 2022-03-11 16:26:01
22 2022-03-15 04:27:52 2022-03-15 10:01:46
23 2022-03-18 05:40:2

In [17]:
variants_for_rows = []
new_intervals = intervals.copy()

for idx, row in intervals.iterrows():
    start_date = row['start']
    end_date = row['finish']
    m1_vairiants = []
    new_intervals.at[idx, 'Unnamed: 0'] = idx
    for column in joined_m1_resampled.columns:
        subset = joined_m1_resampled.loc[start_date:end_date, column]
        count = subset.sum()
        if count > 0:
            machine = column[2:19]
            tech_mest = column[20:]
            m1_vairiants.append((count, machine, tech_mest))
    if m1_vairiants:
        m1_vairiants = list(reversed(sorted(m1_vairiants)))
        machine = m1_vairiants[0][1]
        tm = m1_vairiants[0][2]
        result_tuple = (idx, start_date, end_date, machine, tm)
        variants_for_rows.append(result_tuple)
        new_intervals.at[idx, 'tm'] = tm
        new_intervals.at[idx, 'machine'] = machine
        print(result_tuple)

    # добавить в dataframe



(44, Timestamp('2022-04-29 13:01:08'), Timestamp('2022-04-29 16:14:38'), 'ЭКСГАУСТЕР А/М №8', 'РОТОР ЭКСГ. №8')
(46, Timestamp('2022-04-30 18:06:28'), Timestamp('2022-04-30 23:28:32'), 'ЭКСГАУСТЕР А/М №6', 'ЗАДВИЖКА ЭКСГ. №6')
(70, Timestamp('2022-06-20 12:03:48'), Timestamp('2022-06-20 17:40:32'), 'ЭКСГАУСТЕР А/М №8', 'ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №8')
(138, Timestamp('2023-01-02 06:01:18'), Timestamp('2023-01-02 10:03:08'), 'ЭКСГАУСТЕР А/М №6', 'ЗАДВИЖКА ЭКСГ. №6')
(139, Timestamp('2023-01-07 12:25:37'), Timestamp('2023-01-07 17:26:54'), 'ЭКСГАУСТЕР А/М №8', 'РОТОР ЭКСГ. №8')
(140, Timestamp('2023-01-08 18:05:08'), Timestamp('2023-01-08 23:28:50'), 'ЭКСГАУСТЕР А/М №8', 'РОТОР ЭКСГ. №8')
(162, Timestamp('2023-03-03 20:38:12'), Timestamp('2023-03-04 00:36:29'), 'ЭКСГАУСТЕР А/М №7', 'РОТОР ЭКСГ. №7')
(163, Timestamp('2023-03-04 11:29:22'), Timestamp('2023-03-04 15:41:25'), 'ЭКСГАУСТЕР А/М №7', 'РОТОР ЭКСГ. №7')
(170, Timestamp('2023-03-11 00:37:24'), Timestamp('2023-03-11 05:07:

In [19]:
new_intervals.to_excel('submission_1.xlsx')

In [16]:
row['Unnamed: 0']

69